# Understanding the Glen-Stokes Model for Glacier Flow

## 1. Introduction to Glacier Flow
Glaciers are massive bodies of ice that flow slowly under their own weight. The flow of ice in a glacier is a complex process governed by physical laws that describe how the ice deforms and moves.
Given an initial glacier geometry, the time evolution in ice thickness $h(x, y, t)$ is determined by the mass conservation equation, which couples ice dynamics and surface mass balance (SMB)
through: 

$
\begin{aligned}
    \frac{\partial h}{\partial t} + \nabla \cdot (\mathbf{u}h) = SMB,
     \\
\end{aligned}
$

where $\nabla \cdot$ denotes the divergence operator with respect to horizontal variables $(x, y)$, $\mathbf{u}$ is the vertically averaged horizontal ice velocity field and SMB the SMB function, which consists of the
integration of ice accumulation and ablation over one year. Equation (1) is generic and can be applied to model glacier evolution in number of applications provided adequate SMB and ice-flow model components. 
In the following, we mostly focus on developing an efficient numerical method to compute the ice-flow considering it is often the most computationally expensive component in glacier evolution model.

- First, we will use a numerical solution to compute $\mathbf{u}$. 
- Then we will use a data-driven Machine Learning (ML) approach that emulates ice-flow (Jouvet and others, 2022). 
The state of the art ML techniques ( Jouvet and Cordonnier, 2023), use a Physics Informed Neural Network (PINN); however, this goes beyond the scope of this chapter which focuses on hybrid models.






## 2. The Glen-Stokes Equations
The **Glen-Stokes model** is a widely used theoretical framework to describe this flow, particularly in the context of ice dynamics.

### 2.1 Stokes Equations for Glacier Flow
The Stokes equations are fundamental to describing the motion of viscous fluids, including glacier ice. For glaciers, the flow is generally assumed to be slow enough that inertial forces can be neglected, leaving us with the simplified Stokes equations:

$
\begin{aligned}
    -\nabla \cdot \sigma &= \rho \mathbf{g}, \\
    \nabla \cdot \mathbf{u} &= 0.
\end{aligned}
$

Where:
- $\sigma$ is the Cauchy stress tensor, which represents internal stresses within the glacier ice.
- $\rho$ is the density of ice (about 910 kg/m³).
- $\mathbf{g}$ is the gravitational acceleration vector.
- $\mathbf{u}$ is the velocity vector of the ice.

### 2.2 Glen's Flow Law
Glen's flow law relates the strain rate (deformation rate) of the ice to the applied stress. This relationship is crucial for understanding how ice deforms under pressure and is given by:

$
\mathbf{D}(\mathbf{u}) = \frac{1}{2} (\nabla \mathbf{u} + \nabla \mathbf{u}^\top)
$

The stress-strain relationship, often called Glen's law, is expressed as:

$
\mathbf{\tau} = 2\eta \mathbf{D}(\mathbf{u}),
$

Where:
- $\mathbf{\tau}$ is the deviatoric stress tensor (a measure of stress causing deformation).
- $\eta$ is the effective viscosity of ice, which depends on the temperature and the stress applied to the ice.
- $\mathbf{D}(\mathbf{u})$ is the strain rate tensor.

Glen's law is non-linear and can be expressed in the following form:

$
\mathbf{\tau} = 2 A^{-1/n} \left|\mathbf{D}(\mathbf{u})\right|^{\frac{1-n}{n}} \mathbf{D}(\mathbf{u}),
$

Where:
- $A$ is the flow rate factor, dependent on temperature.
- $n$ is Glen’s exponent, typically taken as 3 for glacier ice, indicating the non-linear relationship between stress and strain rate.
- $\left|\mathbf{D}(\mathbf{u})\right|$ is the magnitude of the strain rate tensor.



### 2.3. Transition to the Ice Sheet Equation

The Glen-Stokes equations describe the flow of ice under stress, but their computational cost makes them impractical for large-scale ice-sheet modeling over long time periods. This leads us to introduce the **shallow ice approximation (SIA)**, which is used in ice sheet models due to its simplicity and efficiency.

The SIA simplifies the full Stokes equations by assuming that the ice flow is dominated by vertical shear stresses and that horizontal stresses can be neglected. This results in an expression for the ice velocity in terms of the ice thickness gradient and the surface slope. The ice velocity $\mathbf{u}$ can be approximated as:

$
\mathbf{u} = -\left(\frac{2 A}{n+2}\right) \left(\rho g \sin(s)\right)^n h^{n+1} \nabla h,
$

where:
- $\rho$ is the ice density,
- $g$ is the gravitational acceleration,
- $A$ is Glen's flow rate factor,
- $n$ is Glen's law exponent (typically 3),
- $h$ is the ice thickness, and
- $s$ is the surface slope.

This equation governs the horizontal velocity of ice based on the local ice thickness and slope, and is more computationally feasible than the full Glen-Stokes model.
We plug the **u** into the mass conservation equation and obtain: 

$
\frac{\partial h}{\partial t} + \nabla \cdot (D(h,z)\frac{\partial z}{\partial x}) = \text{SMB},
$

where $D(h,z) =  f_d (\rho g)^3 h^5 |\nabla S|^2$.

This equation describes the time evolution of the ice thickness, where the velocity $\mathbf{u}$ is computed from the ice sheet's surface slope and thickness. This approach provides a balance between accuracy and computational efficiency and is widely used in large-scale ice sheet models.

Below is the implementation of this equation in Python:


## 3. Boundary Conditions

### 3.1 No-Slip Condition at the Base
In many glacier models, we assume a no-slip condition at the base, meaning the ice velocity is zero at the bedrock. This condition is suitable for glaciers frozen to their beds:

$
\mathbf{u} = 0 \text{ on the bedrock surface}.
$

### 3.2 Stress-Free Surface
At the glacier surface (exposed to air), a stress-free boundary condition is typically applied:

$
\sigma \cdot \mathbf{n} = 0 \text{ on the surface},
$

Where $\mathbf{n}$ is the outward normal vector at the glacier surface.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
from scipy import ndimage

In [ ]:

# Physical parameters
Lx = 80000    # Domain length in x (m)
Ly = 85000    # Domain length in y (m)
ttot = 7000   # Time limit (yr)
grad_b = 0.01 # Mass balance gradient (no unit)
b_max = 0.1   # Maximum precip (m/yr)
Z_ELA = 2500  # Elevation of equilibrium line altitude (m)
rho = 910.0   # Ice density (g/m^3)
g   = 9.81    # Earth's gravity (m/s^2)
fd  = 1e-18   # Deformation constant (Pa^-3 y^-1) 


# Numerical parameters
nx = 100  # Number of cells in x
ny = 100  # Number of cells in y


# Initialization & load data
Z_topo = np.loadtxt('mte_rosa.dat')
Z_topo = ndimage.zoom(Z_topo, (ny/Z_topo.shape[0], nx/Z_topo.shape[1]), order=1) 
 
nout = 10000  # Frequency of plotting
dtmax = 1   # maximum time step
dt = dtmax  # Initial time step
dx = Lx / (nx - 1)  # Cell size in x
dy = Ly / (ny - 1)  # Cell size in y
x = np.linspace(0, Lx, nx)  # x-coordinates
y = np.linspace(0, Ly, ny)  # y-coordinates


H_ice = np.zeros((ny, nx))  # Initial ice thickness
Z_surf = Z_topo + H_ice  # Initial ice surface
time = 0  # Initial time
it = 0

In [ ]:

# Loop
while time < ttot:
    
    # Update time
    time += dt
    it   += 1


    # Calculate H_avg, size (ny-1,nx-1)
    H_avg = 0.25 * (H_ice[:-1, :-1] + H_ice[1:, 1:] + H_ice[:-1, 1:] + H_ice[1:, :-1])


    # Compute Snorm, size (ny-1,nx-1)
    Sx = np.diff(Z_surf, axis=1) / dx
    Sy = np.diff(Z_surf, axis=0) / dy
    Sx = 0.5 * (Sx[:-1, :] + Sx[1:, :])
    Sy = 0.5 * (Sy[:, :-1] + Sy[:, 1:])
    Snorm = np.sqrt(Sx**2 + Sy**2)


    # Compute D, size (ny-1,nx-1)
    D = fd * (rho * g)**3.0 * H_avg**5 * Snorm**2


    # Compute dt
    dt = min(min(dx, dy)**2 / (4.1 * np.max(D)), dtmax)


    # Compute qx, size (ny-2,nx-1)
    qx = -(0.5 * (D[:-1,:] + D[1:,:])) * np.diff(Z_surf[1:-1,:], axis=1) / dx


    # Compute qy, size (ny-1,nx-2)
    qy = -(0.5 * (D[:,:-1] + D[:,1:])) * np.diff(Z_surf[:,1:-1,], axis=0) / dy


    # Update rule (diffusion)
    dHdt = -(np.diff(qx, axis=1) / dx + np.diff(qy, axis=0) / dy)
    H_ice[1:-1, 1:-1] += dt * dHdt # size (ny-2,nx-2)


    b = np.minimum(grad_b * (Z_surf - Z_ELA), b_max)


    # Update rule (mass balance)
    H_ice[1:-1, 1:-1] += dt * b[1:-1, 1:-1]


    # Update rule (positive thickness)
    H_ice = np.maximum(H_ice, 0)


    # updatesurface topography
    Z_surf = Z_topo + H_ice
  
    # Change ELA after 5000 years
    if time > 5000:
        Z_ELA = 2900

    # Display
    if it % nout == 0:
        plt.figure(2, figsize=(11, 4),dpi=200)
        plt.subplot(1, 2, 1)
        plt.imshow(Z_surf, extent=[0, Lx/1000, 0, Ly/1000],  cmap='terrain',origin='lower')
        plt.colorbar(label='Elevation (m)')
        plt.title('Ice Surface at ' + str(int(time))+ ' y') 
        plt.xlabel('Distance, km') 
        plt.ylabel('Distance, km') 


        plt.subplot(1, 2, 2)
        plt.imshow(np.where(H_ice>0,H_ice,np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet',origin='lower')
        plt.colorbar(label='Ice Thickness (m)')
        plt.title('Ice Thickness at ' + str(int(time))+ ' y') 
        plt.xlabel('Distance, km') 
        plt.ylabel('Distance, km') 
        plt.show()



## 4. Machine Learning implementation

In this section we will train an ML model to emulate an ice flow model. 
The ML model is data driven, the data is generated from First Order Approximation of the Glen-Stokes law. 
This model is trained to predict **u**(u,v) given $\{ h(x,y), \frac{\partial s}{\partial x},\frac{\partial s}{\partial y}, c \}$. 
The input and output fields are 2 D grid rasters: 

$
\R^{N_x \times N_y \times 4} \rightarrow \R^{N_x \times N_y \times 2} ,
$

where $N_x, N_y$ are the dimensions in x and y axis.